In [ ]:
import pandas as pd
from fuc import common

In [ ]:
sv_genes = common.convert_file2list('data/sv-genes.list')
df = pd.read_csv('data/combined.csv')

sv_df = df[df.Gene.isin(sv_genes)]
sv_df = sv_df.pivot(index='Sample', columns='Gene', values='CNV')

mappings = {x+1: f'0{x+1}' if x < 9 else str(x+1) for x in range(10)}
batch_df = df[df.Gene == 'ABCB1'][['Sample', 'Batch']]
batch_df.Batch = batch_df.Batch.replace(mappings)
batch_dict = dict(zip(batch_df.Sample, batch_df.Batch))

In [ ]:
base = ''

def one_sv(x, sample, batch):
    sv, gene = x.split(',')
    return f'[{sv}]({base}/{gene}/{batch}/{sample}_NYGC.png)'

def one_row(r):
    genes = r.index.to_series()
    r = r.str.cat(genes, sep=',')
    batch = batch_dict[r.name]
    r = r.apply(one_sv, args=(r.name, batch))
    return r
    
sv_df = sv_df.apply(one_row, axis=1)
sv_df.to_csv('test.csv')

In [ ]:
df = pd.read_csv('sv-table.csv')
with open('sv-table.md', 'w') as f:
    f.write(df.to_markdown(tablefmt='github'))